In [1]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
except:
  # We are not in Google Colab
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

# 1.0 The Manipulator Hessian

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part II: Acceleration and Advanced Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[1.1 Prelude](#pre)
* Part I of the Tutorial
  * Deriving the Manipulator Jacobian
  * First Derivative of an Elementary Transform
  * The Manipulator Jacobian
  * Fast Manipulator Jacobian

[1.2 The Manipulator Hessian](#hess)
* The Manipulator Hessian

[1.3 Fast Manipulator Hessian](#fhess)
* Fast Manipulator Hessian

[1.4 Changing Hessian Frame](#frame)

[1.5 Robotics Toolbox Jacobians](#rtb)

[1.6 Speed Analysis](#fast)



In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm
import spatialmath.base as smb

# use timeit to benchmark some methods
from timeit import default_timer as timer

# ansitable is a great package for printing tables in a terminal
from ansitable import ANSITable

<br>

<a id='pre'></a>
### 1.1 Prelude
---


We begin with the forward kineamtics of a manipulator as described in Part I

\begin{align*}
    {^0\bf{T}_e(t)}  &= {\cal K}(\bf{q}(t)) \\
    &= \prod_{i=1}^{M} \bf{E}_i(\eta_i).
 \end{align*}

where $\bf{q}(t) \in \mathbb{R}^n$ is the vector of joint generalised coordinates, $n$ is the number of joints, and $M$ is the number of elementary transforms $\bf{E}_i \in \bf{SE}(3)$. From the derivative of this we can express the spatial translational and angular velocity as a function of the joint coordinates and velocities

\begin{align*}
    \bf{\nu} =
    \begin{pmatrix}
        \bf{v} \\ \bf{\omega}
    \end{pmatrix}
    &=
    \bf{J}(\bf{q})\dot{\bf{q}}
\end{align*}

where $\bf{J}(\bf{q})$ is the manipulator Jacobian, $\bf{v} = (v_x, \ v_y, \ v_z)$, and $\bf{\omega} =(\omega_x, \ \omega_y, \ \omega_z)$. Taking the temporal derivative gives

\begin{align*}
    \begin{pmatrix}
        \dot{\bf{v}} \\ \dot{\bf{\omega}}
    \end{pmatrix}
    &=
    \begin{pmatrix}
        \bf{a} \\ \bf{\alpha}
    \end{pmatrix}
    =
    \dot{\bf{J}} \dot{\bf{q}} + \bf{J} \ddot{\bf{q}}
\end{align*}

where $\bf{a} \in \mathbb{R}^3$ is the end-effector translational acceleration, $\bf{\alpha} \in \mathbb{R}^3$ is the end-effector angular acceleration, and

\begin{align*}
    \dot{\bf{J}}
    &=
    \dfrac{\mathrm{d} \bf{J}(\bf{q})}
        {\mathrm{d} t}  \\
    &=
    \dfrac{\partial \bf{J}(\bf{q})}
        {\partial q_1} \dot{q}_1
    +
    \dfrac{\partial \bf{J}(\bf{q})}
        {\partial q_2} \dot{q}_2
    + \cdots +
    \dfrac{\partial \bf{J}}
        {\partial q_n} \dot{q}_n \\
    &=
    \begin{pmatrix}
        \dfrac{\partial \bf{J}(\bf{q})}
            {\partial q_1} &
        \dfrac{\partial \bf{J}(\bf{q})}
            {\partial q_2} &
        \cdots &
        \dfrac{\partial \bf{J}(\bf{q})}
            {\partial q_n}
    \end{pmatrix}
    \dot{\bf{q}} \\
    &=
    \bf{H}(\bf{q}) \dot{\bf{q}} \\
    &=
    \begin{pmatrix}
        \bf{H}_a(\bf{q}) \\ \bf{H}_\alpha(\bf{q})
    \end{pmatrix} \dot{\bf{q}}
\end{align*}

where $\bf{H}(\bf{q}) \in \mathbb{R}^{6 \times n \times n}$ in the manipulator Hessian tensor, which is the partial derivative of the manipulator Jacobian with respect to the joint coordinates, $\bf{H}_a(\bf{q}) \in \mathbb{R}^{3 \times n \times n}$ forms the translational component of the Hessian, and $\bf{H}_\alpha(\bf{q}) \in \mathbb{R}^{3 \times n \times n}$ forms the angular component of the Hessian.

From Part I of the Tutorial, we the derivative of an elementary transform with respect to a joint coordinate is obtained using one of

\begin{align*}
    \dfrac{\mathrm{d} \bf{T}_{R_{x}}(\theta)}
        {\mathrm{d} \theta}
    &=     
    \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & -1 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix} \bf{T}_{R_{x}}(\theta),\\
    \dfrac{\mathrm{d} \bf{T}_{R_{y}}(\theta)}
        {\mathrm{d} \theta}
    &=     
    \begin{pmatrix}
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 \\
        -1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix} \bf{T}_{R_{y}}(\theta), \\
    \dfrac{\mathrm{d} \bf{T}_{R_{z}}(\theta)}
        {\mathrm{d} \theta}
    &=     
    \begin{pmatrix}
        0 & -1 & 0 & 0 \\
        1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0  
    \end{pmatrix} \bf{T}_{R_{z}}(\theta),
\end{align*}

for a revolute joint, or one of

\begin{align*}
    \dfrac{\mathrm{d} \bf{T}_{t_{x}}(d)}
        {\mathrm{d} d}
    &=     
    \begin{pmatrix}
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix}, \\
    \dfrac{\mathrm{d} \bf{T}_{t_{y}}(d)}
        {\mathrm{d} d}
    &=     
    \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix}, \\
    \dfrac{\mathrm{d} \bf{T}_{t_{z}}(d)}
        {\mathrm{d} d}
    &=     
    \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 
    \end{pmatrix},
 \end{align*}

for a prismatic joint.

In [3]:
# We can make Python functions which perform these derivatives

def dTRx(θ, flip):
    '''
    Calculates the derivative of an SE3 which is a pure rotation around
    the x-axis by amount θ
    '''
    Rhx = np.array([
        [0, 0, 0, 0],
        [0, 0, -1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_x matrix in the maths above
    Trx = smb.trotx(θ)

    if flip:
        return Rhx.T @ Trx.T
    else:
        return Rhx @ Trx

def dTRy(θ, flip):
    '''
    Calculates the derivative of an SE3 which is a pure rotation around
    the y-axis by amount θ
    '''
    Rhy = np.array([
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [-1, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_y matrix in the maths above
    Try = smb.troty(θ)

    if flip:
        return Rhy.T @ Try.T
    else:
        return Rhy @ Try

def dTRz(θ, flip):
    '''
    Calculates the derivative of an SE3 which is a pure rotation around
    the z-axis by amount θ
    '''
    Rhz = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_z matrix in the maths above
    Trz = smb.trotz(θ)

    if flip:
        return Rhz.T @ Trz.T
    else:
        return Rhz @ Trz

def dTtx():
    '''
    Calculates the derivative of an SE3 which is a pure translation along
    the x-axis by amount d
    '''
    Thx = np.array([
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    return Thx

def dTty():
    '''
    Calculates the derivative of an SE3 which is a pure translation along
    the y-axis by amount d
    '''
    Thy = np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    return Thy

def dTtz():
    '''
    Calculates the derivative of an SE3 which is a pure translation along
    the z-axis by amount d
    '''
    Thz = np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0]
    ])

    return Thz

Now we make a method which can take the derivative of an ET

In [4]:
def dET(et, η):
    '''
    This method takes an ET and returns the derivative with respect to the joint coordinate
    This is here for convenience
    '''

    if et.axis == 'Rx':
        return dTRx(η, et.isflip)
    elif et.axis == 'Ry':
        return dTRy(η, et.isflip)
    elif et.axis == 'Rz':
        return dTRz(η, et.isflip)
    elif et.axis == 'tx':
        return dTtx()
    elif et.axis == 'ty':
        return dTty()
    else:
        return dTtz()

For the second derivative of an elementary transform with respect to the same joint variable, the result is
%
\begin{align*}
    \dfrac{\mathrm{d}^2 \bf{T}_{R_{x}}(\theta)}
        {\mathrm{d} \theta^2}
    &=
    \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & -1 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix}
    \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & -1 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix} \bf{T}_{R_{x}}(\theta),\\
    \dfrac{\mathrm{d}^2 \bf{T}_{R_{y}}(\theta)}
        {\mathrm{d} \theta^2}
    &=     
    \begin{pmatrix}
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 \\
        -1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix}
    \begin{pmatrix}
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 \\
        -1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix} \bf{T}_{R_{y}}(\theta), \\
    \dfrac{\mathrm{d}^2 \bf{T}_{R_{z}}(\theta)}
        {\mathrm{d} \theta^2}
    &=     
    \begin{pmatrix}
        0 & -1 & 0 & 0 \\
        1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0  
    \end{pmatrix}
    \begin{pmatrix}
        0 & -1 & 0 & 0 \\
        1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0  
    \end{pmatrix} \bf{T}_{R_{z}}(\theta),
\end{align*}

for a revolute joint, or a zero matrix for a prismatic joint.

In [5]:
# We can make Python functions which perform these derivatives

def ddTRx(θ, flip):
    '''
    Calculates the second derivative of an SE3 which is a pure rotation around
    the x-axis by amount θ
    '''
    Rhx = np.array([
        [0, 0, 0, 0],
        [0, 0, -1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0]
    ])

    print(θ)

    # This is the T_R_x matrix in the maths above
    Trx = smb.trotx(θ)

    if flip:
        return Rhx @ Rhx @ Trx.T
    else:
        return Rhx @ Rhx @ Trx

def ddTRy(θ, flip):
    '''
    Calculates the second derivative of an SE3 which is a pure rotation around
    the y-axis by amount θ
    '''
    Rhy = np.array([
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [-1, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_y matrix in the maths above
    Try = smb.troty(θ)

    if flip:
        return Rhy @ Rhy @ Try.T
    else:
        return Rhy @ Rhy @ Try

def ddTRz(θ, flip):
    '''
    Calculates the second derivative of an SE3 which is a pure rotation around
    the z-axis by amount θ
    '''
    Rhz = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_z matrix in the maths above
    Trz = smb.trotz(θ)

    if flip:
        return Rhz @ Rhz @ Trz.T
    else:
        return Rhz @ Rhz @ Trz

def ddTtx():
    '''
    Calculates the second derivative of an SE3 which is a pure translation along
    the x-axis by amount d
    '''
    return np.zeros((4, 4))

def ddTty():
    '''
    Calculates the second derivative of an SE3 which is a pure translation along
    the y-axis by amount d
    '''
    return np.zeros((4, 4))

def ddTtz():
    '''
    Calculates the second derivative of an SE3 which is a pure translation along
    the z-axis by amount d
    '''
    return np.zeros((4, 4))

Now we make a method which can take the derivative of an ET

In [6]:
def ddET(et, η):
    '''
    This method takes an ET and returns the derivative with respect to the joint coordinate
    This is here for convenience
    '''

    if et.axis == 'Rx':
        return ddTRx(η, et.isflip)
    elif et.axis == 'Ry':
        return ddTRy(η, et.isflip)
    elif et.axis == 'Rz':
        return ddTRz(η, et.isflip)
    elif et.axis == 'tx':
        return ddTtx()
    elif et.axis == 'ty':
        return ddTty()
    else:
        return ddTtz()

In [7]:
# We need the following utility methods (see Notebook 1 of Part I)
def vex(mat):
    '''
    Converts a 3x3 skew symmetric matric to a 3 vector
    '''

    return np.array([mat[2, 1], mat[0, 2], mat[1, 0]])

def ρ(tf):
    '''
    The method extracts the rotational component from an SE3
    '''
    return tf[:3, :3]

def τ(tf):
    '''
    The method extracts the translation component from an SE3
    '''
    return tf[:3, 3]

We also need the first derivative of an ETS (see Notebook 1 of Part I)

In [8]:
def dT_j(ets, j, q):
    '''
    This methods calculates the dervative of an SE3 with respect to joint coordinate j
    '''

    # Allocate an identity matrix for the result
    dT = np.eye(4)

    # Find the jth variable et in the ets
    et_j = ets.joints()[j]

    # Loop overs the ETs in the robot ETS
    for et in ets:
        if et == et_j:
            # This ET is variable and corresponds to joint j
            dT = dT @ dET(et, q[et.jindex])
        elif et.isjoint:
            # This ET is a variable joint
            # Use the q array to specify the joint angle for the variable ET
            dT = dT @ et.A(q[et.jindex])
        else:
            # This ET is static
            dT = dT @ et.A()

    return dT

In [9]:
# Before going any further, lets remake the ETS panda model we made in Notebook 1 of Part I

E1 = rtb.ET.tz(0.333) 
E2 = rtb.ET.Rz()
E3 = rtb.ET.Ry() 
E4 = rtb.ET.tz(0.316) 
E5 = rtb.ET.Rz()
E6 = rtb.ET.tx(0.0825) 
E7 = rtb.ET.Ry(flip=True) 
E8 = rtb.ET.tx(-0.0825) 
E9 = rtb.ET.tz(0.384) 
E10 = rtb.ET.Rz()
E11 = rtb.ET.Ry(flip=True)
E12 = rtb.ET.tx(0.088) 
E13 = rtb.ET.Rx(np.pi / 2) 
E14 = rtb.ET.tz(0.107)
E15 = rtb.ET.Rz()
panda = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13 * E14 * E15

# And make a joint coordinate array q
q = np.array([0, -0.3, 0, -2.2, 0, 2, 0.79])

<br>

<a id='hess'></a>
### 1.2 The Manipulator Hessian
---


The second partial derivative of a pose with respect to the joint variables $q_j$ and $q_k$,
can be obtained by taking the derivative of

\begin{align*}
    \frac{\partial \bf{T}(\bf{q})}
        {\partial q_j} 
    &=
    \frac{\partial} 
        {\partial q_j}  
    \left(
        \bf{E}_1(\eta_1) \bf{E}_2(\eta_2) \cdots \bf{E}_M(\eta_M) 
    \right)\\
    &= 
    \prod_{i=1}^{\mu(j)-1} \bf{E}_i(\eta_i) 
    \frac{\mathrm{d} \bf{E}_{\mu(j)}(q_j)} 
        {\mathrm{d} q_j} 
    \prod_{i=\mu(j)+1}^{M} \bf{E}_i(\eta_i).
\end{align*}

with respect to $q_k$. This results in the following, where the function $\mu(j)$ returns the index in the ETS where $q_j$ appears as a variable

\begin{align*}
    \frac{\partial^2 \bf{T}}
        {\partial q_k q_j} &= 
    \begin{pmatrix} 
        \bf{H}_{R_{j_k}} & \bf{H}_{t_{j_k}} \\ 
        0  & 0 
    \end{pmatrix}  \\
    &= 
    \frac{\partial} 
        {\partial q_k} 
    \left( 
    \prod_{i=1}^{\mu(j)-1} \bf{E}_i(\eta_i)
    \
    \frac{\mathrm{d} \bf{E}_{\mu(j)}(q_j)} 
        {d q_j} 
    \
    \prod_{i=\mu(j)+1}^{M} \bf{E}_i(\eta_i)  
    \right)   \\
    &= \left\{ 
    \begin{matrix}
        \prod\limits_{i=1}^{\mu(k) - 1} \bf{E}_i(\eta_i)
        \
        \frac{\mathrm{d} \bf{E}_{\mu(k)}(q_k)}
            {d q_k}
        \
        \prod\limits_{i=\mu(k)+1}^{\mu(j) - 1} \bf{E}_i(\eta_i) 
        \
        \frac{\mathrm{d} \bf{E}_{\mu(j)}(q_j)}
            {d q_j} 
        \
        \prod\limits_{i=\mu(j)+1}^{M} \bf{E}_i(\eta_i)
        & \text{if} \ k < j \\
        \prod\limits_{i=1}^{\mu(k) - 1} E_i(\eta_i) 
        \
        \frac{\mathrm{d}^2 \bf{E}_{\mu(k)}(q_k)}
            {d q_k^2} 
        \
        \prod\limits_{i=\mu(k)+1}^{M} E_i(\eta_i) 
        & \text{if} \ k = j \\
        \prod\limits_{i=1}^{\mu(j) - 1} \bf{E}_i(\eta_i) 
        \
        \frac{\mathrm{d} \bf{E}_{\mu(j)}(q_j)}
            {d q_j}
        \
        \prod\limits_{i=\mu(j)+1}^{\mu(k) - 1} \bf{E}_i(\eta_i) 
        \
        \frac{\mathrm{d} \bf{E}_{\mu(k)}(q_k)}
            {d q_k} 
        \
        \prod\limits_{i=\mu(k)+1}^{M} \bf{E}_i(\eta_i)
        & \text{if} \ k > j \\
    \end{matrix}
    \right.
\end{align*}

In the above equation, we can fing the derivative of an elementary transform with respect to a joint coordinate using the method described in Section 1.1 above.

For the second derivative of an elementary transform with respect to the same joint variable, as is the case for $k = j$ in the above equation, the result is a zero matrix.

Now we make a method which can take the second derivative of an a pose as described in the equation above.

In [10]:
def ddT_jk(ets, j, k, q):
    '''
    This methods calculates the second dervative of an SE3 with
    respect to joint coordinate j and k
    '''

    # Allocate an identity matrix for the result
    ddT = np.eye(4)

    # Find the jth variable et in the ets
    et_j = ets.joints()[j]

    # Find the kth variable et in the ets
    et_k = ets.joints()[k]

    # Loop overs the ETs in the robot ETS
    for et in ets:

        if et == et_j and j == k:
            # Case 2 in the above equation
            ddT = ddT @ ddET(et, q[et.jindex])
        elif et == et_j:
            # Case 1 or 3 in the above equation
            # This ET is variable and corresponds to joint j
            ddT = ddT @ dET(et, q[et.jindex])
        elif et == et_k:
            # Case 1 or 3 in the above equation
            # This ET is variable and corresponds to joint j
            ddT = ddT @ dET(et, q[et.jindex])
        elif et.isjoint:
            # This ET is a variable joint
            # Use the q array to specify the joint angle for the variable ET
            ddT = ddT @ et.A(q[et.jindex])
        else:
            # This ET is static
            ddT = ddT @ et.A()

    return ddT

As for the manipulator Jacobian, to form the manipulator Hessian, we partition it into translational and rotational components.

To form $\bf{H}_{\alpha_{jk}}$, the angular component of the manipulator Hessian of a joint variable $j$ with respect to another joint variable $k$, we take the partial derivative of the $j^{th}$ column of the manipulator Jacobian in 

\begin{align*}
    \bf{J}_{\omega_j}(\bf{q}) 
    =
    \mathsf{V}_\times
    \left(
        \rho
        \left(
            \frac{\partial \bf{T}(\bf{q})}
                {\partial q_j} 
        \right)
        \rho
        \left(
            \bf{T}(\bf{q})
        \right)^\top
    \right)
\end{align*}

using the product rule

\begin{align*}
    \bf{H}_{\omega_{jk}}
    &=
    \dfrac{\partial \bf{J}_{\omega_j})(\bf{q})}
        {\partial q_k} \\
    &=
    \vee_\times \bigg(
        \rho
        \left(
            \frac{\partial^2 \bf{T}(\bf{q})}
                {\partial q_j \partial q_k} 
        \right)
        \rho
        \left(
            \bf{T}(\bf{q})
        \right)^\top
    \bigg.
    + 
    \bigg.
        \rho
        \left(
            \frac{\partial \bf{T}(\bf{q})}
                {\partial q_j} 
        \right)
        \rho
        \left(
            \frac{\partial \bf{T}(\bf{q})}
                {\partial q_k} 
        \right)^\top
    \bigg) \\
    &=
    \mathsf{V}_\times
    \left(
        \bf{H}_{R_{jk}} \
        \rho
        \left(
            \bf{T}(\bf{q})
        \right)^\top
        +
        \bf{J}_{R_j}
        \bf{J}_{R_k}^\top
    \right)
\end{align*}

where $\bf{H}_{\omega_{jk}} \in \mathbb{R}^3$, and $\bf{H}_{R_{jk}}$ is obtained from above.



In [11]:
def Hω_jk(ets, j, k, q):
    '''
    This method calculates the rotational component of the jth
    slice, kth column of the manipulator Hessian
    '''

    # Calculate the forward kinematics at q
    T = ets.eval(q)

    # Calculate the second derivative of T with respect to q_j, q_k
    ddT = ddT_jk(ets, j, k, q)

    # Calculate the derivative of T with respect to q_j
    dTj = dT_j(ets, j, q)

    # Calculate the derivative of T with respect to q_k
    dTk = dT_j(ets, k, q)

    Hω = vex(ρ(ddT) @ (ρ(T).T) + ρ(dTj) @ (ρ(dTk).T))

    return Hω

In [12]:
# Calculate the angular component of the third slice, second column of Panda manipulator Hessian
print(Hω_jk(panda, 2, 1, q))

[9.55336489e-01 5.96560643e-33 2.95520207e-01]


To form $\bf{H}_{a_{jk}}$, the translational component of the manipulator Hessian for joint variable $j$ with respect to another joint variable $k$, we take the partial derivative of the $j^{th}$ translational component of the manipulator Jacobian in

\begin{align*}
    \bf{J}_{v_{j}}(\bf{q}) 
    &=
    \tau
    \left(
        \frac{\partial \bf{T}(\bf{q})}
            {\partial q_j} 
    \right)
\end{align*}

which provides

\begin{align*}
    \bf{H}_{v_{jk}}
    &=
    \dfrac{\partial \bf{J}_{v_{j}}(\bf{q})}
        {\partial q_k} \\
    &=
    \tau
    \left(
        \frac{\partial^2 \bf{T}(\bf{q})}
            {\partial q_j \partial q_k} 
    \right) \\
    &= 
    \bf{H}_{t_{jk}}
\end{align*}

where $\bf{H}_{a_{jk}} \in \mathbb{R}^3$, and $\bf{H}_{t_{jk}}$ is obtained from above.

In [13]:
def Hv_jk(ets, j, k, q):
    '''
    This method calculates the translation component of the jth
    slice, kth column of the manipulator Hessian
    '''

    # Calculate the derivative of T with respect to q_j
    ddT = ddT_jk(ets, j, k, q)

    Hv = τ(ddT)

    return Hv

In [14]:
# Calculate the translation component of the third slice, second column of Panda manipulator Hessian
print(Hv_jk(panda, 2, 1, q))

[ 0.03162066  0.         -0.102221  ]


Stacking the translational and angular components, we form the component of the manipulator Hessian for joint variable $j$ with respect to another joint variable $k$

\begin{align*}
    \bf{H}_{jk}
    &=
    \begin{pmatrix}
        \bf{H}_{v_{jk}} \\
        \bf{H}_{\omega_{jk}}
    \end{pmatrix}
\end{align*}

where $\bf{H}_{jk} \in \mathbb{R}^{6}$.

The component of the manipulator Hessian for joint variable $j$ is formed by arranging the abpve into columns of a matrix

\begin{align*}
    \bf{H}_j
    &=
    \begin{pmatrix}
        \bf{H}_{j1} &
        \cdots &
        \bf{H}_{jn}
    \end{pmatrix}
\end{align*}

where $\bf{H}_{j} \in \mathbb{R}^{6 \times n}$.

The whole manipulator Hessian is formed by arranging the above into _slices_ of a tensor

\begin{align*}
    \bf{H}
    &=
    \begin{pmatrix}
        \bf{H}_1 &
        \cdots &
        \bf{H}_n
    \end{pmatrix}
\end{align*}

where $\bf{H} \in \mathbb{R}^{n \times 6 \times n}$ and last last two dimensions of $\bf{H}$ define the dimension of the slices $\bf{H}_i$.

<br>

<br>

<img src="img/hessian_dark.png" alt="drawing" width="900"/>

_Visualisation of the Hessian $\bf{H}(\bf{q})$ representing a 7-joint manipulator. Each slice of the Hessian $\bf{H}_i(\bf{q})$ represents the acceleration of the end-effector caused by the velocities of each joint $\bf{q}$ with respect to the velocity of joint $q_i$. Within a slice, the top three rows $\bf{H}_{\alpha_i}$ correspond to the linear acceleration, while the bottom three rows $\bf{H}_{a_i}$ correspond to the angular acceleration, of the end-effector $\bf{\alpha}$ caused by the velocities of the joints._

In [15]:
# By using our previously defined methods, we can now calculate the manipulator Hessian

def hessian0(ets, q):
    '''
    This method calculates the manipulator Hessian in the world frame
    '''

    # Allocate array for the Hessian
    # It is a nx6xn matrix
    H = np.zeros((ets.n, 6, ets.n))

    for j in range(ets.n):
        for k in range(ets.n):
            Hv = Hv_jk(ets, j, k, q)
            Hω = Hω_jk(ets, j, k, q)

            H[k, :3, j] = Hv
            H[k, 3:, j] = Hω

    return H

In [16]:
# Calculate the manipulator Hessian of the Panda in the world frame
H = hessian0(panda, q)

print(np.round(H, 2))


[[[-0.46  0.   -0.53  0.    0.04  0.    0.  ]
  [ 0.11  0.29  0.1   0.04 -0.03 -0.01  0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.   -1.   -0.    1.    0.    1.    1.  ]
  [ 0.    0.   -0.3   0.    0.95  0.   -0.  ]
  [-0.   -0.   -0.   -0.   -0.   -0.    0.  ]]

 [[ 0.   -0.46  0.03  0.48 -0.1   0.09  0.  ]
  [ 0.29  0.    0.    0.    0.    0.    0.  ]
  [ 0.   -0.29 -0.1  -0.04  0.03  0.01  0.  ]
  [ 0.    0.    0.96  0.   -0.32  0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.   -0.  ]
  [-0.    0.    0.3   0.   -0.95  0.    0.  ]]

 [[-0.53  0.03 -0.5   0.    0.03  0.    0.  ]
  [ 0.1   0.    0.11  0.18 -0.06  0.02  0.  ]
  [ 0.   -0.1  -0.16  0.    0.01  0.    0.  ]
  [ 0.    0.   -0.    0.96  0.    0.96  0.96]
  [ 0.    0.    0.   -0.    0.81 -0.    0.  ]
  [-0.    0.   -0.    0.3   0.    0.3   0.3 ]]

 [[ 0.    0.48  0.   -0.48  0.1  -0.09  0.  ]
  [ 0.04  0.    0.18  0.    0.    0.    0.  ]
  [ 0.   -0.04  0.    0.04 -0.03 -0.01  0.  ]
  [ 0.    0.    0.    0.    

<br>
   
<a id='fhess'></a>
### 1.3 Fast Manipulator Hessian
---

Calculatint the manipulator Hessian using the above has $\mathcal{O}(n^3)$ time complexity which we can improve.

We revisit the previous equations while substituting in

\begin{align*}
    \frac{\mathrm{d}\bf{R}(\theta)}
        {\mathrm{d} \theta} 
    &= 
    [
        \hat{\bf{\omega}}
    ]_\times
    \bf{R}(\theta(t)) 
\end{align*}

and simplify

\begin{align*}
    \bf{H}_{\omega_{jk}}
    &=
    \vee_\times \bigg(
        {[\hat{\bf{\omega}}_k]_\times}
        {[\hat{\bf{\omega}}_j]_\times}
        \bf{R}(\bf{q})
        \bf{R}(\bf{q})^\top + 
    \bigg. \\
    & \qquad \
    \bigg.
        {[\hat{\bf{\omega}}_j]_\times}
        \bf{R}(\bf{q})
        \left(
        {[\hat{\bf{\omega}}_k]_\times}
        \bf{R}(\bf{q})
        \right)^\top
    \bigg) \\
    &=
    \vee_\times \bigg(
        {[\hat{\bf{\omega}}_k]_\times}
        {[\hat{\bf{\omega}}_j]_\times} + 
        {[\hat{\bf{\omega}}_j]_\times}
        \bf{R}(\bf{q}) \bf{R}(\bf{q})^\top
        {[\hat{\bf{\omega}}_k]_\times}^\top
    \bigg) \\
    &=
    \vee_\times \bigg(
        {[\hat{\bf{\omega}}_k]_\times}
        {[\hat{\bf{\omega}}_j]_\times} - 
        {[\hat{\bf{\omega}}_j]_\times}
        {[\hat{\bf{\omega}}_k]_\times}
    \bigg)
\end{align*}

Since we know that $\bf{J}_{\omega_x} = {[\hat{\bf{\omega}}_x}]_\times$, and using the identity $[\bf{a} \times \bf{b}]_\times = [\bf{a}]_\times[\bf{b}]_\times-[\bf{b}]_\times[\bf{a}]_\times$ we show that

\begin{align*}
    \bf{H}_{\omega_{jk}}
    &=
    \vee_\times \bigg(
        [\bf{J}_{\omega_{k}}]_\times
        [\bf{J}_{\omega_{j}}]_\times -
        [\bf{J}_{\omega_{j}}]_\times
        [\bf{J}_{\omega_{k}}]_\times
    \bigg)\\
    &= \bf{J}_{\omega_{k}} \times \bf{J}_{\omega_{j}}
\end{align*}

which means that the rotational component of the manipulator Hessian can be calculated from the rotational components of the manipulator Jacobian. 
A key relationship is that the velocity of joint $j$, with respect to the velocity of the same, or preceding joint $k$, does not contribute acceleration to the end-effector from the perspective of joint $j$. Consequently, $\bf{H}_{\omega_{jk}}=0$ when $k\geq j$.

For the translational component of the manipulator Hessian $\bf{H}_v$, we can see in (\ref{eq:ddets}) that two of the conditions will have the same result: when $k < j$, and when $k > j$. Therefore, we have

\begin{align*}
    \bf{H}_{\omega_{jk}}(\bf{q}) = \bf{H}_{v_{kj}}(\bf{q})
\end{align*}

and by exploiting this relationship, we can simplify (\ref{eq:hess1}) to

\begin{align*}
    \bf{H}_{\omega_{jk}}(\bf{q})
    &= [\bf{J}_{\omega_{k}}]\times \bf{J}_{v_j} \\
    &= \bf{J}_{\omega_a} \times \bf{J}_{v_b}
\end{align*}

where $a = \min(j,k)$, and $b = \max(j,k)$. This means that the translational component of the manipulator Hessian can be calculated from components of the manipulator Jacobian.

Through this simplification, computation of the manipulator Hessian reduces to $\mathcal{O}(n^2)$ time complexity.

In [17]:
# Now making the efficient version in Python

def hessian0_efficient(ets, q):
    '''
    This method calculates the manipulator Hessian in the world frame using a more efficient method
    '''

    # Make the Jacobian
    J = ets.jacob0(q)

    # Allocate space for the Hessian
    H = np.zeros((ets.n, 6, ets.n))

    for j in range(ets.n):
        for i in range(j, ets.n):

            H[j, :3, i] = np.cross(J[3:, j], J[:3, i])
            H[j, 3:, i] = np.cross(J[3:, j], J[3:, i])

            if i != j:
                H[i, :3, j] = H[j, :3, i]

    return H

In [18]:
H = hessian0_efficient(panda, q)

print(f"The manipulator Hessian (world-frame) \nin configuration q = {q} \nis: \n{np.round(H, 2)}")

The manipulator Hessian (world-frame) 
in configuration q = [ 0.   -0.3   0.   -2.2   0.    2.    0.79] 
is: 
[[[-0.46 -0.   -0.53  0.    0.04  0.    0.  ]
  [ 0.11  0.29  0.1   0.04 -0.03 -0.01  0.  ]
  [ 0.    0.    0.    0.   -0.    0.    0.  ]
  [ 0.   -1.    0.    1.   -0.    1.    1.  ]
  [ 0.   -0.   -0.3   0.    0.95  0.   -0.  ]
  [ 0.    0.    0.   -0.    0.   -0.   -0.  ]]

 [[-0.   -0.46  0.03  0.48 -0.1   0.09  0.  ]
  [ 0.29 -0.    0.    0.   -0.    0.    0.  ]
  [ 0.   -0.29 -0.1  -0.04  0.03  0.01 -0.  ]
  [ 0.    0.    0.96  0.   -0.32  0.    0.  ]
  [ 0.    0.    0.    0.   -0.    0.    0.  ]
  [ 0.    0.    0.3   0.   -0.95  0.    0.  ]]

 [[-0.53  0.03 -0.5   0.    0.03  0.    0.  ]
  [ 0.1   0.    0.11  0.18 -0.06  0.02  0.  ]
  [ 0.   -0.1  -0.16 -0.    0.01  0.   -0.  ]
  [ 0.    0.    0.    0.96 -0.    0.96  0.96]
  [ 0.    0.    0.    0.    0.81  0.    0.  ]
  [ 0.    0.    0.    0.3  -0.    0.3   0.3 ]]

 [[ 0.    0.48  0.   -0.48  0.1  -0.09 -0.  ]
  [ 0.04  

<br>

<a id='frame'></a>
### 1.4 Changing Hessian Frame
---

We have been calculating the manipulator Hessian in the world-frame

\begin{equation*}
     {^0\bf{H}}(\bf{q}).
\end{equation*}

is the manipulator Hessian matrix expressed in the world-coordinate frame.

The Hessian expressed in the end-effector frame is

\begin{equation*}
    {^e\bf{H}}(\bf{q}) =
    \begin{pmatrix} 
        {{^0\bf{R}}_e^\top} & \bf{0} \\ 
        \bf{0} & {{^0\bf{R}}_e^\top}
     \end{pmatrix}
     {^0\bf{H}}(\bf{q})
\end{equation*}

where ${^0\bf{R}}_e$ is a rotation matrix representing the orientation of the end-effector in the world frame. And we perform the matrix multiplication along slices of the tensor ${^0\bf{H}}(\bf{q})$.


In [19]:
# Now making it in Python

def hessiane(ets, q):
    '''
    This method calculates the manipulator Hessian in the end-effector frame
    '''

    # Allocate the return array
    He = np.zeros((ets.n, 6, ets.n))

    # Calculate world-frame Hessian
    H0 = hessian0_efficient(ets, q)

    # Calculate the forward kineamtics
    T = ets.eval(q)

    # Extract rotation from T and transpose
    R = T[:3, :3].T

    # Make our velocity transform matrix
    tr = np.zeros((6, 6))

    # Put the rotation in the top left and bottom right of the matrix
    tr[:3, :3] = R
    tr[3:, 3:] = R

    # Perform the matrix multiplication to calculate the Hessian
    # in the end-effector frame
    for i in range(ets.n):
        He[i] = tr @ H0[i]

    return He
    

In [20]:
# Calculate the manipulator Hessian of the Panda in the end-effector frame
H = hessiane(panda, q)

print(f"The manipulator Hessian (end-effector) \nin configuration q = {q} \nis: \n{np.round(H, 2)}")

The manipulator Hessian (end-effector) 
in configuration q = [ 0.   -0.3   0.   -2.2   0.    2.    0.79] 
is: 
[[[-0.29  0.   -0.33  0.    0.02 -0.    0.  ]
  [ 0.36  0.    0.41  0.   -0.03 -0.    0.  ]
  [-0.11 -0.29 -0.1  -0.04  0.03  0.01  0.  ]
  [ 0.   -0.63 -0.    0.63  0.    0.63  0.63]
  [ 0.    0.78 -0.   -0.78  0.   -0.78 -0.78]
  [ 0.    0.    0.3  -0.   -0.95 -0.    0.  ]]

 [[ 0.   -0.52 -0.06  0.27 -0.04  0.06  0.  ]
  [ 0.    0.18 -0.09 -0.39  0.1  -0.06  0.  ]
  [-0.29 -0.   -0.   -0.    0.    0.    0.  ]
  [ 0.    0.    0.83  0.   -0.94  0.    0.  ]
  [ 0.    0.   -0.56  0.   -0.34  0.   -0.  ]
  [ 0.    0.    0.    0.   -0.    0.    0.  ]]

 [[-0.33 -0.06 -0.44  0.    0.03  0.    0.  ]
  [ 0.41 -0.09  0.29  0.   -0.02  0.    0.  ]
  [-0.1  -0.   -0.11 -0.18  0.06 -0.02  0.  ]
  [ 0.    0.    0.    0.83  0.    0.83  0.83]
  [ 0.    0.    0.   -0.56  0.   -0.56 -0.56]
  [ 0.    0.    0.    0.   -0.81  0.   -0.  ]]

 [[ 0.    0.27  0.   -0.27  0.04 -0.06  0.  ]
  [ 0.   

<br>

<a id='rtb'></a>
### 1.5 Robotics Toolbox Hessians
---

The `ETS` class within the Robotics Toolbox supports many kinematic methods (such as the previously discussed `.eval` and `.fkine`, `jacob0`, and `jacobe` methods).

The `.hessian0` and `.hessiane` methods in the `ETS` class calculate the world and end-effector frame Hessians respectively.


In [21]:
# Note: The panda object which we have been using is an instance of the ETS class

# Calculate the world frame Hessian using the ETS class
H0 = panda.hessian0(q)

# Calculate the end-effector frame Hessian using the ETS class
He = panda.hessiane(q)

# View our Hessians
print(f"The manipulator Hessian (world frame) is: \n{np.round(H0, 2)}")
print(f"\nThe manipulator Hessian (end-effector frame) is: \n{np.round(He, 2)}")

The manipulator Hessian (world frame) is: 
[[[-0.46 -0.   -0.53  0.    0.04  0.    0.  ]
  [ 0.11  0.29  0.1   0.04 -0.03 -0.01  0.  ]
  [ 0.    0.    0.    0.   -0.    0.    0.  ]
  [ 0.   -1.    0.    1.   -0.    1.    1.  ]
  [ 0.   -0.   -0.3   0.    0.95  0.   -0.  ]
  [ 0.    0.    0.   -0.    0.   -0.   -0.  ]]

 [[-0.   -0.46  0.03  0.48 -0.1   0.09  0.  ]
  [ 0.29 -0.    0.    0.   -0.    0.    0.  ]
  [ 0.   -0.29 -0.1  -0.04  0.03  0.01 -0.  ]
  [ 0.    0.    0.96  0.   -0.32  0.    0.  ]
  [ 0.    0.    0.    0.   -0.    0.    0.  ]
  [ 0.    0.    0.3   0.   -0.95  0.    0.  ]]

 [[-0.53  0.03 -0.5   0.    0.03  0.    0.  ]
  [ 0.1   0.    0.11  0.18 -0.06  0.02  0.  ]
  [ 0.   -0.1  -0.16 -0.    0.01  0.   -0.  ]
  [ 0.    0.    0.    0.96 -0.    0.96  0.96]
  [ 0.    0.    0.    0.    0.81  0.    0.  ]
  [ 0.    0.    0.    0.3  -0.    0.3   0.3 ]]

 [[ 0.    0.48  0.   -0.48  0.1  -0.09 -0.  ]
  [ 0.04  0.    0.18 -0.    0.   -0.    0.  ]
  [ 0.   -0.04 -0.    0.04 -0.0

<br>

<a id='fast'></a>
### 1.6 Speed Comparison
---

In this section we will briefly cover a speed comparison between the two `hessian0` methods shown here and the Hessian implementation provided by the Robotics Toolbox.

Kinematic methods in the toolbox are written using C extensions and wrapped by python methods. These C methods also have Python fallbacks to rely on if something goes wrong.

_This takes around 30 seconds on a decent desktop computer_

In [22]:
# Number of iterations to test over
n = 1000

# Make a random matrix of q values
qt = np.random.random((n, 7))

# Slow Hessian implementation
start = timer()
for q in qt:
    hessian0(panda, q)
slow_time = timer() - start

# Improved Hessian implementation
start = timer()
for q in qt:
    hessian0_efficient(panda, q)
improved_time = timer() - start

# Toolbox C improved Hessian implementation
start = timer()
for q in qt:
    panda.hessian0(q)
rtb_time = timer() - start

# Make a table
print(f"\nTiming to Calculate Manipulator Hessian over {n} iterations\n")
table = ANSITable(
    "Function",
    "Total Time (ms)",
    "Average Time (us)",
    border="thin",
)

table.row(
    "our hessian0",
    np.round(slow_time * 1000.0, 4),
    np.round((slow_time / n) * 1000000.0, 4),
)

table.row(
    "our hessian0_efficient",
    np.round(improved_time * 1000.0, 4),
    np.round((improved_time / n) * 1000000.0, 4),
)

table.row(
    "toolbox hessian0",
    np.round(rtb_time * 1000.0, 4),
    np.round((rtb_time / n) * 1000000.0, 4),
)
table.print()


Timing to Calculate Manipulator Hessian over 1000 iterations

┌───────────────────────┬─────────────────┬───────────────────┐
│              Function │ Total Time (ms) │ Average Time (us) │
├───────────────────────┼─────────────────┼───────────────────┤
│          our hessian0 │      15823.0887 │        15823.0887 │
│our hessian0_efficient │        638.9757 │          638.9757 │
│      toolbox hessian0 │           1.007 │             1.007 │
└───────────────────────┴─────────────────┴───────────────────┘

